In [1]:
from firedrake import *
from irksome import RadauIIA, Dt, MeshConstant, TimeStepper, PEPRK, GaussLegendre, ContinuousPetrovGalerkinScheme
from scipy.io import loadmat
from scipy.interpolate import RegularGridInterpolator

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


In [2]:
# butcher_tableau = RadauIIA(2)
# butcher_tableau = PEPRK(4, 2, 5)
# butcher_tableau = GaussLegendre(3)
butcher_tableau = ContinuousPetrovGalerkinScheme(2)
# ns = butcher_tableau.num_stages

In [3]:
Nx = 50
Ny = 50
L = 1.5
h = 1
# msh = PeriodicRectangleMesh(Nx, Ny, L, h, direction='x', quadrilateral=True)
msh = RectangleMesh(Nx, Ny, L, 1)#, quadrilateral=True)

In [4]:
# Create periodic mesh and function spaces
msh_p = PeriodicRectangleMesh(Nx, Ny, L, h, direction='x')#, quadrilateral=True)
V = VectorFunctionSpace(msh_p, "CG", 2, dim=2)
W = FunctionSpace(msh_p, "CG", 1)
Z = V*W

MC = MeshConstant(msh_p)
# dt = 0.002
dt = MC.Constant(0.025)
t = MC.Constant(0.0)
Re = MC.Constant(190.0)

In [5]:
def velocity_interpolator_1(x, interp1):
    pts = np.column_stack((x[1], x[0]))
    u1 = interp1(pts)
    return u1

def velocity_interpolator_2(x, interp2):
    pts = np.column_stack((x[1], x[0]))
    u2 = interp2(pts)
    return u2

def import_velocity(mat_data, u):
    X2d = mat_data['X']
    Y2d = mat_data['Y'] + 0.5 # shift y -> [0, 1]

    x = X2d[0, :] 
    y = Y2d[:, 0] 

    interp1 = RegularGridInterpolator((y, x), mat_data['u1_values'], bounds_error=False, fill_value=0.0)
    interp2 = RegularGridInterpolator((y, x), mat_data['u2_values'], bounds_error=False, fill_value=0.0)

    V1 = FunctionSpace(msh, "CG", 2) 

    xcoord = Function(V1)
    ycoord = Function(V1)

    xcoord.project(msh.coordinates[0])
    ycoord.project(msh.coordinates[1])


    coords_dofs = np.column_stack((xcoord.dat.data, ycoord.dat.data))

    # Build (y,x) points for MATLAB data
    pts = np.column_stack((coords_dofs[:, 1], coords_dofs[:, 0]))  # (y, x)

    # Evaluate data
    u1_vals = interp1(pts)
    u2_vals = interp2(pts)

    # Put into scalar CG1 functions
    u1_cg1 = Function(V1, name="u1_cg1")
    u2_cg1 = Function(V1, name="u2_cg1")

    u1_cg1.dat.data[:] = u1_vals
    u2_cg1.dat.data[:] = u2_vals

    # Project initial condition from non-periodic mesh to periodic mesh
    V1_p = FunctionSpace(msh_p, "CG", 2) 

    u1 = Function(V1_p, name="u1")
    u1.project(u1_cg1)

    u2 = Function(V1_p, name="u2")
    u2.project(u2_cg1)

    V1_vec = VectorFunctionSpace(msh_p, "CG", 2, dim=2) 
    u_cg1 = Function(V1_vec, name="u_cg1")
    u_cg1.project(as_vector([u1, u2]))

    # u0 = Function(V, name="Velocity_IC")  
    u.project(u_cg1)

    return u

In [6]:
# Import initial condition
mat_data = loadmat("Data_couette/lambda0.020078Re190L1.5.mat")
u0 = Function(V, name="Velocity_IC")
u0 = import_velocity(mat_data, u0)

# Import eigenfunctions
mat_data = loadmat("Data_couette/EF1_Re190L1.5.mat")
u1 = Function(V, name="Velocity_EF1")
u1 = import_velocity(mat_data, u1)

mat_data = loadmat("Data_couette/EF2_Re190L1.5.mat")
u2 = Function(V, name="Velocity_EF2")
u2 = import_velocity(mat_data, u2)

mat_data = loadmat("Data_couette/EF3_Re190L1.5.mat")
u3 = Function(V, name="Velocity_EF3")
u3 = import_velocity(mat_data, u3)

mat_data = loadmat("Data_couette/EF4_Re190L1.5.mat")
u4 = Function(V, name="Velocity_EF4")
u4 = import_velocity(mat_data, u4)

mat_data = loadmat("Data_couette/EF5_Re190L1.5.mat")
u5 = Function(V, name="Velocity_EF5")
u5 = import_velocity(mat_data, u5)

mat_data = loadmat("Data_couette/EF6_Re190L1.5.mat")
u6 = Function(V, name="Velocity_EF6")
u6 = import_velocity(mat_data, u6)


In [7]:
# Couette laminar flow in [0, 1]
U = Function(V)
x, y = SpatialCoordinate(msh_p)
psi = -0.5 + y

U.assign(project(as_vector([psi, 0]), V))

Coefficient(WithGeometry(FunctionSpace(<firedrake.mesh.MeshTopology object at 0x7e687f1036a0>, VectorElement(FiniteElement('Lagrange', triangle, 2), dim=2), name=None), Mesh(VectorElement(FiniteElement('Discontinuous Lagrange', triangle, 1, variant='equispaced'), dim=2, variant='equispaced'), 6)), 643)

In [8]:
# Get pressure from velocity initial condition
def get_pressure_ic():
    P = FunctionSpace(msh_p, "CG", 1)
    p0 = Function(P, name="Pressure_IC")
    q = TestFunction(P)
    p_trial = TrialFunction(P)

    dx_p = dx(msh_p)
    
    f = - inner(dot(grad(u0), u0), grad(q)) * dx_p - inner(dot(grad(U), u0), grad(q)) * dx_p - inner(dot(grad(u0), U), grad(q)) * dx_p
    a = inner(grad(p_trial), grad(q)) * dx_p

    nsp = VectorSpaceBasis(constant=True, comm=msh_p.comm)
    
    solve(a == f, p0, nullspace=nsp)
    return p0

p0 = get_pressure_ic()
# p0 = Function(W)

In [9]:
# Initialize solution function
up = Function(Z)
u_, p_ = up.subfunctions
u_.rename('Velocity')
p_.rename("Pressure")

# Assign initial conditions
u_.assign(u0)
p_.assign(p0)

# Write initial conditions to file
out = File("Results/initial_couette.pvd")
out.write(u_, p_)

# Set up variational problem
dx_p = dx(msh_p)

u, p = split(up)
v, w = TestFunctions(Z)
F = (inner(Dt(u), v) * dx_p + inner(dot(grad(u), u), v) * dx_p
     + inner(dot(grad(U), u), v) * dx_p + inner(dot(grad(u), U), v) * dx_p
     + 1/Re * inner(grad(u), grad(v)) * dx_p - inner(p, div(v)) * dx_p
     + inner(div(u), w) * dx_p)

bcs = [DirichletBC(Z.sub(0), as_vector([0, 0]), (1, 2))]
E = inner(u_, u_) * dx_p
a1 = inner(u_, u1)*dx_p
a2 = inner(u_, u2)*dx_p
a3 = inner(u_, u3)*dx_p
a4 = inner(u_, u4)*dx_p
a5 = inner(u_, u5)*dx_p
a6 = inner(u_, u6)*dx_p

# Solver parameters
luparams = {"mat_type": "aij",
            "snes_type": "newtonls",
            "snes_monitor": None,
            "ksp_type": "preonly",
            "pc_type": "lu",
            "pc_factor_mat_solver_type": "mumps",
            "snes_linesearch_type": "l2",
            "snes_force_iteration": 1,
            "snes_rtol": 1e-5,
            "snes_atol": 1e-5
            }

# Define nullspace for pressure
nsp = MixedVectorSpaceBasis(Z, [Z.sub(0), VectorSpaceBasis(constant=True, comm=msh_p.comm)])

stepper = TimeStepper(F, butcher_tableau, t, dt, up, bcs=bcs,
                      solver_parameters=luparams, nullspace=nsp)

energy = []
functional = []
times = []
out = File("ns_Couette_Re190_v.pvd")
out.write(u_, p_, time=float(t))

et = assemble(E)
a_1 = float(assemble(a1))
a_2 = float(assemble(a2))
a_3 = float(assemble(a3))
a_4 = float(assemble(a4))
a_5 = float(assemble(a5))
a_6 = float(assemble(a6))
v = Function(V)
v.assign(u_ - a_1*u1 - a_2*u2 - a_3*u3 - a_4*u4 - a_5*u5 - a_6*u6)
q_e = np.sqrt(assemble(inner(v, v)*dx_p))
P_eval = 0.133299895117 * q_e**2 + 0.134856691579 * a_6**2 + 0.134856691579 * a_5**2 - 0.010918103575 * a_4 * a_6 + 0.000317381157411 * a_4 * a_5 + 0.133448043454 * a_4**2 - 0.000317381157411 * a_3 * a_6 - 0.010918103575 * a_3 * a_5 + 0.133448043454 * a_3**2 + 0.112694597481 * a_2 * q_e**2 + 0.110939746857 * a_2 * a_6**2 + 0.110939746857 * a_2 * a_5**2 - 0.002183684911 * a_2 * a_4 * a_6 + 0.000175089975731 * a_2 * a_4 * a_5 + 0.117180744306 * a_2 * a_4**2 - 0.000175089975731 * a_2 * a_3 * a_6 - 0.002183684911 * a_2 * a_3 * a_5 + 0.117180744306 * a_2 * a_3**2 + 0.133168477035 * a_2**2 + 0.0756768821866 * a_2**3 + 3.01740234273e-05 * a_1 * q_e**2 + 2.62080363393e-05 * a_1 * a_6**2 + 2.62080363393e-05 * a_1 * a_5**2 + 8.57424778571e-06 * a_1 * a_4 * a_6 + 1.22604162553e-05 * a_1 * a_4 * a_5 + 1.94199708991e-05 * a_1 * a_4**2 - 1.22604162553e-05 * a_1 * a_3 * a_6 + 8.57424778571e-06 * a_1 * a_3 * a_5 + 1.94199708991e-05 * a_1 * a_3**2 + 1.26858049681e-05 * a_1 * a_2 + 3.40257514013e-05 * a_1 * a_2**2 + 0.133375817258 * a_1**2 + 0.0829512968455 * a_1**2 * a_2 + 2.08419622779e-05 * a_1**3
func_eval = et**2 + P_eval

print(f"  Energy = {0.5*et}")
print(f" Functional = {func_eval}")
energy.append(0.5*et)
functional.append(func_eval)
times.append(float(t))

# Time stepping loop
for _ in range(400):
     print(f"Stepping from time {float(t)}")
     stepper.advance()
     t.assign(float(t) + float(dt))
     et = assemble(E)
     a_1 = float(assemble(a1))
     a_2 = float(assemble(a2))
     a_3 = float(assemble(a3))
     a_4 = float(assemble(a4))
     a_5 = float(assemble(a5))
     a_6 = float(assemble(a6))
     v = Function(V)
     v.assign(u_ - a_1*u1 - a_2*u2 - a_3*u3 - a_4*u4 - a_5*u5 - a_6*u6)
     q_e = np.sqrt(assemble(inner(v, v)*dx_p))
     P_eval = 0.133299895117 * q_e**2 + 0.134856691579 * a_6**2 + 0.134856691579 * a_5**2 - 0.010918103575 * a_4 * a_6 + 0.000317381157411 * a_4 * a_5 + 0.133448043454 * a_4**2 - 0.000317381157411 * a_3 * a_6 - 0.010918103575 * a_3 * a_5 + 0.133448043454 * a_3**2 + 0.112694597481 * a_2 * q_e**2 + 0.110939746857 * a_2 * a_6**2 + 0.110939746857 * a_2 * a_5**2 - 0.002183684911 * a_2 * a_4 * a_6 + 0.000175089975731 * a_2 * a_4 * a_5 + 0.117180744306 * a_2 * a_4**2 - 0.000175089975731 * a_2 * a_3 * a_6 - 0.002183684911 * a_2 * a_3 * a_5 + 0.117180744306 * a_2 * a_3**2 + 0.133168477035 * a_2**2 + 0.0756768821866 * a_2**3 + 3.01740234273e-05 * a_1 * q_e**2 + 2.62080363393e-05 * a_1 * a_6**2 + 2.62080363393e-05 * a_1 * a_5**2 + 8.57424778571e-06 * a_1 * a_4 * a_6 + 1.22604162553e-05 * a_1 * a_4 * a_5 + 1.94199708991e-05 * a_1 * a_4**2 - 1.22604162553e-05 * a_1 * a_3 * a_6 + 8.57424778571e-06 * a_1 * a_3 * a_5 + 1.94199708991e-05 * a_1 * a_3**2 + 1.26858049681e-05 * a_1 * a_2 + 3.40257514013e-05 * a_1 * a_2**2 + 0.133375817258 * a_1**2 + 0.0829512968455 * a_1**2 * a_2 + 2.08419622779e-05 * a_1**3
     func_eval = et**2 + P_eval
     print(f"  Energy = {0.5*et}")
     print(f" Functional = {func_eval}")
     times.append(float(t))
     energy.append(0.5*et)
     functional.append(func_eval)

     out.write(u_, p_, time=float(t))

/home/vicente/NavierStokesEnergy/venv-firedrake/lib/python3.10/site-packages/firedrake/_deprecation.py:65: UserWarning: The use of `File` for output is deprecated, please update your code to use `VTKFile` from `firedrake.output`.
  warn(
/home/vicente/NavierStokesEnergy/venv-firedrake/lib/python3.10/site-packages/firedrake/_deprecation.py:65: UserWarning: The use of `File` for output is deprecated, please update your code to use `VTKFile` from `firedrake.output`.
  warn(


  Energy = 0.500027828847458
 Functional = 1.1335664609079303
Stepping from time 0.0
  0 SNES Function norm 5.025352004143e-04
  1 SNES Function norm 9.274020645060e-07
  Energy = 0.5005231944424003
 Functional = 1.1310046442716577
Stepping from time 0.025
  0 SNES Function norm 5.988268940397e-04
  1 SNES Function norm 2.058897922878e-06
  Energy = 0.500971878223935
 Functional = 1.1283453621725796
Stepping from time 0.05
  0 SNES Function norm 5.945658725548e-04
  1 SNES Function norm 2.064224437600e-06
  Energy = 0.5013262115324003
 Functional = 1.1254291878475353
Stepping from time 0.07500000000000001
  0 SNES Function norm 5.898120982311e-04
  1 SNES Function norm 2.062358501036e-06
  Energy = 0.5015450840341035
 Functional = 1.1221234068989476
Stepping from time 0.1
  0 SNES Function norm 5.844202697859e-04
  1 SNES Function norm 2.053395181232e-06
  Energy = 0.5015899717947323
 Functional = 1.1183029353434815
Stepping from time 0.125
  0 SNES Function norm 5.783255668940e-04
  1

In [10]:
np.savez('CouetteRe190L1.5_v', arr1=energy, arr2=functional, arr3=times)